# Quadratic binary optimization for problems with inequality constraints
Based on the [Constrained Quadratic Model](https://docs.ocean.dwavesys.com/en/stable/concepts/cqm.html) - CQM.

# 1 Theory
The `cost function` $C_f(\boldsymbol{x})$ can be represented as
\begin{equation}
C_f(\boldsymbol{x})=\sum_{i} c_{ii} x_i + \sum_{i < j} c_{ij} x_i x_j + const,
\label{eq:costcqm}
\end{equation} where $c_{ii}$ and $c_{ij}$ are real coefficients, and the variables $\{ x_i\}_{i=1, \dots, N}$ are binary.
---
<br> Similarly, the `constraints` are given by functions $G^{(k)}_{f}(\boldsymbol{x})$ can be represented as 
\begin{equation}
 G^{(k)}_{f}(\boldsymbol{x})=\sum_i g^{(k)}_{ii} x_i + \sum_{i < j} g^{(k)}_{ij} x_i x_j+ const \circ 0, 
 \label{eq:conscqm}
 \end{equation}
    where $g_{ii}$ and $g_{ij}$ are real coefficients, $k=1, \dots, M$ and $M$ is the number of constraints, and the symbol $\circ$ denotes a comparison operator  $\{\ge, \le, =\}.$
---    

<br> To use QAOA, all functions have to be combined into Quantum Unconstrained Binary Optimization. In the first step, this is done by transforming the constraints $G^{(k)}_{f}(\boldsymbol{x}) \circ 0 $ into the equality constraints $K^{(k)}_{f}(\boldsymbol{x})=0$. This is done using slack variables. 

---  
<br> Next, by adding weighted constraints and the cost function together, the `objective function` is obtained in the form
\begin{equation}
\label{qubo}
    f_{QUBO}(\boldsymbol{x})=\alpha_0C_f(\boldsymbol{x})+\sum_{k=1}^{M} {\alpha_{k} K_f^{(k)}(\boldsymbol{x})},
\end{equation}
where $\alpha_0$ is the weight of the cost function and $\alpha_{k}$ is the weight of the $k$-th constraint, $\alpha_i > 0$.

# 2 Simplified example
_Minimize $2x_0 + 5x_1 + x_0x_1$ subject to $x_0 + x_1 = 1 $ and $5x_0 + 2x_1 \leq 5 $_

Using the notation introduced in [1]
<center>
minimize $ C_f(\boldsymbol{x}) = 2x_0 + 5x_1 + x_0x_1$
    <br> subject to $G_f^{(1)}(\boldsymbol{x}) = x_0 + x_1 - 1 = 0$ and $G_f^{(2)}(\boldsymbol{x})=5x_0 + 2x_1 - 5 \leq 0 $

We need to convert the inequality constraint $G_f^{(2)}$ into the equality constraint $K_f^{(2)}$ using slack variables. To see how this is done, see the method `calc_slack_coefficients`. In our case $5$ can be expressed as $2^0 + 2^1 + 2$, so the coefficients for binary variables $\textbf{s}$ are $[1, 2, 2]$. In general, if the number is not a power of 2, the last coefficient is the difference between the previous power of 2 that falls within this number and this number.

<br>
<center>
$K_f^{(1)}(\boldsymbol{x}) = x_0 + x_1 - 1$ and $K_f^{(2)}(\boldsymbol{x})=5x_0 + 2x_1 - 1x_2 - 2x_3 -  2x_4 = 0 $

In QHyper, this problem needs to be a subclass of the [Problem class](https://github.com/qc-lab/QHyper/blob/tmek/improve-tutorials/QHyper/problems/base.py).

In [1]:
import numpy as np
import sympy
from sympy.core.expr import Expr

from QHyper.problems.base import Problem
from QHyper.hyperparameter_gen.parser import Expression

class SimpleProblem(Problem):
    
    def __init__(self) -> None:
        self.slack_coefficients = calc_slack_coefficients(5)
        self.variables = sympy.symbols(' '.join([f'x{i}' for i in range(2)]) + ' ' 
                                       + ' '.join([f'x{i+2}' for i in range(len(self.slack_coefficients))]))
        self._set_objective_function()
        self._set_constraints()
        
    def _set_objective_function(self) -> None:
        C_f = 2 * self.variables[0] + 5 * self.variables[1] + self.variables[0] * self.variables[1]
        self.objective_function = Expression(C_f)
        
    def _set_constraints(self):
        K_f1 = self.variables[0] + self.variables[1] - 1
        
        K_f2 = 5 * self.variables[0] + 2 * self.variables[1]
        for i, coefficient in enumerate(self.slack_coefficients):
            K_f2 += coefficient * self.variables[i+2]
            
        self.constraints = [Expression(K_f1), Expression(K_f2)]
    
    #########################
    def get_score(self, result):
        # "10000"
        # this function is used to evaluate the quality of the result
        
        x = [int(val) for val in result]
        
        if x[0] + x[1] -1 == 0 and 5*x[0] + 2*x[1] + 1*x[2] + 2*x[3] + 2*x[4] == 0:
            return 2 * x[0] + 5 * x[1] + x[0] * x[1] 
        
        return None
    #########################

In [2]:
def calc_slack_coefficients(constant: int) -> list[int]:
    num_slack = int(np.floor(np.log2(constant)))
    slack_coefficients = [2 ** j for j in range(num_slack)]
    if constant - 2 ** num_slack >= 0:
        slack_coefficients.append(constant - 2 ** num_slack + 1)
    return slack_coefficients

In [11]:
calc_slack_coefficients(5)

[1, 2, 2]

In [3]:
problem = SimpleProblem()

In [4]:
print(f"Variables used to describe objective function"
      f" and constraints: {problem.variables}")
print(f"Objective function: {problem.objective_function}")
print("Constraints (RHS == 0):")
for constraint in problem.constraints:
    print(f"    {constraint}")

Variables used to describe objective function and constraints: (x0, x1, x2, x3, x4)
Objective function: x0*x1 + 2*x0 + 5*x1
Constraints (RHS == 0):
    x0 + x1 - 1
    5*x0 + 2*x1 + x2 + 2*x3 + 2*x4


# 3 Using QHyper

Initialization part
<br> 'angles' -  QAOA angles - first we have gammas (for the cost Hamiltonian), then we have betas (for the mixer)
<br> 'hyper_args': [1, 1, 1], #  those are the alpha values from Section 1 - Theory

In [16]:
params_cofing = {
        'angles': [[0.5]*5, [0.5]*5], # QAOA angles - first we have gammas (for the cost Hamiltonian), then we have betas (for the mixer)
        'hyper_args': [1, 1, 1], #  those are the alpha values from [1]
    }

In [17]:
# Simple quantum circuit without optimzers will be used to test the results
# WF-QAOA is choosen becasue this PQC has most suitable evaluation function
from QHyper.solvers.vqa.base import VQA
tester_config = {
    'pqc': {
        'type': 'wfqaoa',
        'layers': 5,
    }
}

tester = VQA(problem, config=tester_config)

In [18]:
# Create a VQA instance with HQAOA as PQC and scipy optimizer
# This can be done in two various way
# 1. Providing dict with config (usefull to save experiment confing in e.g JSON)

solver_config = {
    'pqc': {
        'type': 'qaoa',
        'layers': 5,
        'mixer': 'pl_x_mixer',
            
    },
     'optimizer': {
        'type': 'scipy',
        'maxfun': 200,
    },
}
vqa = VQA(problem, config=solver_config)

In [19]:
# 2. Providing actual isntance of each class like VQA and Optimizer
# from QHyper.solvers.vqa.pqc.qaoa import QAOA
# from QHyper.optimizers import ScipyOptimizer

# vqa = VQA(problem, QAOA(layers=5, mixer='pl_x_mixer'), ScipyOptimizer(maxfun=10))

In [20]:
best_params = vqa.solve(params_cofing)
print(f"Best params: {best_params}")

39.4598970087436
39.459893128326215
39.459894607312584
39.45989458656453
39.459896102875945
39.459902363290304
39.45989697842392
39.45989716143159
39.45989675492358
39.45989730162761
39.459896863503694
49.74999999999846
49.7499765443104
49.74999999999846
49.74999999999846
49.74999999999846
49.74999999999845
49.74999999999841
49.74999999999848
49.74999999999843
49.74999999999848
49.74999999999843
58.3815406365339
58.38153183341561
58.38154600530228
58.38154218875553
58.38154553390038
58.38154424945207
58.381540794351714
58.38154012663236
58.3815406231088
58.381540795122575
58.38154060702886
58.71141439398441
58.71141388932949
58.71141001265808
58.711421827119985
58.71141350418472
58.7114151339931
58.71141412909863
58.71141413222003
58.71141405632595
58.71141422969579
58.71141445936476
58.32795380027621
58.32795897448253
58.327966357438015
58.327970825192146
58.32794947247537
58.327950737111394
58.3279542030519
58.32795375676426
58.32795400012082
58.32795344332202
58.32795372122817
58.18

In [21]:
best_results = tester.evaluate(best_params)
print(f"Best results: {best_results}")
print(f"Params used for optimizer:\n{best_params['angles']},\n"
      f"and params used for hyperoptimizer: {best_params['hyper_args']}")

{'01100': 0.15797244728562657, '01011': 0.1338232689607687, '01110': 0.09714368267067784, '01101': 0.09714368267067781, '00001': 0.05782886095810823, '00010': 0.0578288609581082, '00011': 0.05464062332638749, '00100': 0.04946407720544144, '01000': 0.04702778794520923, '10100': 0.036179333146294124, '00110': 0.03288099976636444, '00101': 0.03288099976636443, '00000': 0.025449242285931754, '00111': 0.020943646999113365, '11111': 0.01424918621261374, '11010': 0.008091274423529983, '11001': 0.00809127442352998, '10011': 0.008042533675297643, '11011': 0.0077335522258862015, '10000': 0.007534521058502502, '11100': 0.005234340450924484, '11000': 0.0043378672869913565, '10001': 0.0043109832192873565, '10010': 0.004310983219287333, '01010': 0.004268785550301792, '01001': 0.00426878555030179, '01111': 0.003641119085868947, '10101': 0.0034677092824631434, '10110': 0.003467709282463141, '10111': 0.0033688104266976927, '11110': 0.0021865253404839444, '11101': 0.002186525340483916}
Best results: 0.0

### Versions of the QAOA

1. 'Vanila' QAOA
<img src="imgs/qaoa.png" width="200" height="100">



'pqc': {
        'type': 'qaoa',
        'layers': 3,
        'mixer': 'pl_x_mixer',
        'backend': 'default.qubit',
        },

2. 'Weight-free' WF-QAOA
<img src="imgs/wf-qaoa.png" width="200" height="100">



'pqc': {
        'type': 'wfqaoa',
        'layers': 3,
        'mixer': 'pl_x_mixer',
        'penalty': 0.0
        'backend': 'default.qubit',
        },

3. 'Hyper' H-QAOA
<img src="imgs/hqaoa.png" width="200" height="100">



'pqc': {
        'type': 'hqaoa',
        'layers': 3,
        'mixer': 'pl_x_mixer',
        'backend': 'default.qubit',
        },

### Versions of optimizers

'optimizer': {
        'type': 'scipy',
        'maxfun': 200,
        'bounds': []
    },